---

title: "DAP II:Final Project"
author: "Genevieve Madigan, Summer Negahdar, Jenny Zhong"
date: "Fall 2024"
format: html
execute:
  echo: false
  eval: true

---

## the link to google drive for data

https://drive.google.com/drive/folders/1U2egyK2B7HBMOuB5wlj-HAs8HkjFLrJk?usp=drive_link

# workflow and team members
Genevieve Madigan: #write your github Id here: 
-responsibility: write up and data visualization
Summer Negahdar: Summer99D
-responsibility: creation of shiny app and data visualization
Jenny Zhong: #write your github ID here
-responsibility: data cleaning and preparation



## Introduction and prior articles




## Data Cleaning

To Jenny and Gena: 
things we need to do:
1. upload temp app,
2. save the file as df and as csv file (in the data folder)
3. upload crime rate
4. clean crime df
5. save it as df and as csv (in data folder)
6. merge both dfs based on ????
7. make a scatterplot of crime rate and temperature
8. make a base map (shapely)
9. make a heated map using scatterplot and base map
10. save the final merged csv(in data folder)
11. shiny app
12. (extra credit): use NLP for finding crime-heat (it sohuld not be hard I'd say)


# Load your CSV file
# Crime data: Merging crime data together and merging crime data with ZIP Code
1. Loading data together

In [ ]:
import pandas as pd
import numpy as np
import geopandas as gpd
import shapely
import shiny
from shapely.geometry import Point

In [ ]:
crime2004 = pd.read_csv('Crimes2004.csv')
crime2005 = pd.read_csv('Crimes2005.csv')
crimes67 = pd.read_csv('Crimes20062007.csv')
crimes8910 = pd.read_csv('Crimes20080910.csv')
crimes1112 = pd.read_csv('Crimes20112012.csv')
crimes131415 = pd.read_csv('Crimes201320142015.csv')

2. Examining the columns of crime data 

In [ ]:
print("2004 Columns:", crime2004.columns)
print("2005 Columns:", crime2005.columns)
print("2006 Columns:", crimes67.columns)

3. Merging all datasets together

In [ ]:
totalcrimedata = pd.concat([crime2004, crime2005, crimes67, crimes8910, crimes1112, crimes131415])

4. Examining the merged dataset

In [ ]:
print(totalcrimedata.info())

Whether all years exist

In [ ]:
print(totalcrimedata['Year'].value_counts())

Summary statistics for numerical columns 

In [ ]:
print(totalcrimedata.describe())

In [ ]:
totalcrimedata.to_csv("totalcrimedata.csv", index=False)

Converting latitude and longitude to ZIP Codes

In [ ]:
len(totalcrimedata)

Load ZIP code shapefiles

In [ ]:
zip_shapes = gpd.read_file("/Users/jennyzhong/Documents/GitHub/final-Project-DAP-II/ZIP Code Shapefiles/tl_2015_us_zcta510.shp")

In [ ]:
zip_shapes = zip_shapes.to_crs("EPSG:4326")
print(zip_shapes.head())

crime geometry

In [ ]:
geometry = [Point(xy) for xy in zip(totalcrimedata['Longitude'], totalcrimedata['Latitude'])]

In [ ]:
crime_gdf = gpd.GeoDataFrame(totalcrimedata, geometry=geometry, crs="EPSG:4326") 
print(crime_gdf.head())

In [ ]:
crime_gdf['geometry'] = crime_gdf.geometry.buffer(0.01)

Merging two data sets

In [ ]:
matched_data = gpd.sjoin(crime_gdf, zip_shapes, how="left", predicate="intersects")

matched_data.to_csv("matched_data.csv", index=False)

clean data

In [ ]:
matched_data.drop(columns=['ID', 'District', 'Community Area', 'CLASSFP10', 'MTFCC10', 'FUNCSTAT10', 'ALAND10', 'AWATER10', 'FBI Code', 'Updated On', 'GEOID10'], inplace=True)

operate from CSV file

In [ ]:
matched_data = pd.read_csv("matched_data.csv")
print(matched_data.head())

Drop appropriate columns 

In [ ]:
matched_data.drop(columns=['ID', 'District', 'Community Area', 'CLASSFP10', 'MTFCC10', 'FUNCSTAT10', 'ALAND10', 'AWATER10', 'FBI Code', 'Updated On', 'GEOID10'], inplace=True)
print(matched_data.head())

In [ ]:
matched_data['Date'] = pd.to_datetime(matched_data['Date'], errors='coerce')

In [ ]:
matched_data['YearMonthDay'] = matched_data['Date'].dt.date
matched_data['Time'] = matched_data['Date'].dt.time

In [ ]:
matched_data.drop(columns=['Date', 'Time', 'GEOID10', 'CLASSFP10', 'MTFCC10', 'FUNCSTAT10', 'ALAND10', 'AWATER10', 'Updated On', 'FBI Code'], inplace=True)
matched_data.head(5)

In [ ]:
columns = matched_data.columns.tolist()
columns.remove('YearMonthDay')
columns.insert(columns.index('Case Number') + 1, 'YearMonthDay')
matched_data = matched_data[columns]
print(matched_data.columns)

In [ ]:
matched_data.to_csv("final_crime_data.csv", index=False)

# Merging weather data together and merging weather data with ZIP code 

Merging all datasets together

In [ ]:
weather2004 = pd.read_csv('AT2004.csv')
weather2005 = pd.read_csv('AT2005.csv')
weather2006 = pd.read_csv('AT2006.csv')
weather2007 = pd.read_csv('AT2007.csv')
weather2008 = pd.read_csv('AT2008.csv')
weather2009 = pd.read_csv('AT2009.csv')
weather2010 = pd.read_csv('AT2010.csv')
weather2011 = pd.read_csv('AT2011.csv')
weather2012 = pd.read_csv('AT2012.csv')
weather2013 = pd.read_csv('AT2013.csv')
weather2014 = pd.read_csv('AT2014.csv')
weather2015 = pd.read_csv('AT2015.csv')

In [ ]:
total_weather = pd.concat([
    weather2004, weather2005, weather2006, weather2007, weather2008, 
    weather2009, weather2010, weather2011, weather2012, weather2013, 
    weather2014, weather2015
], ignore_index=True)

total_weather_data.to_csv('merged_weather_data.csv', index=False)

In [ ]:
total_weather.head(20)

convert weather data to geodataframe

In [ ]:
geometry = [Point(xy) for xy in zip(total_weather['LONGITUDE'], total_weather['LATITUDE'])]

weather_gdf = gpd.GeoDataFrame(total_weather, geometry=geometry)

weather_gdf.set_crs("EPSG:4326", inplace=True)

weather_gdf['geometry'] = weather_gdf.geometry.buffer(0.01)

Load the ZIP Code Shapefile

In [ ]:
zip_shapefile = gpd.read_file('/Users/jennyzhong/Documents/GitHub/final-Project-DAP-II/ZIP_Code_Shapefiles/tl_2015_us_zcta510.shp')

zip_shapefile = zip_shapefile.to_crs("EPSG:4326")

Spatial join to match the zip codes

In [ ]:
weather_with_zip = gpd.sjoin(weather_gdf, zip_shapefile, how='left', predicate='intersects')

print(weather_with_zip.head())

drop irrelevant dataframes: 

In [ ]:
weather_with_zip.drop(["CLASSFP10", "MTFCC10", "FUNCSTAT10", "ALAND10", "AWATER10", "GEOID10"], axis=1, inplace=True)

print(weather_with_zip)

final csv file for weather 

In [ ]:
weather_with_zip.to_csv("final_weather_data.csv", index=False)

## Data Visualization


## Shiny App


In [ ]:
I will upload the file:
merged_df= pd.read_csv('/Users/samarnegahdar/Desktop/Final_project/final-Project-DAP-II/Datasets/merged_data.csv')